In [1]:
from osgeo import ogr
from  osgeo import ogr, osr
import os
import pickle

def filterCountyRings(key,union,filter_table=None):
    '''Returns a polygon with largest area in multiPolygon geometry object.'''
    if filter_table is None:
        return union
    if union.GetGeometryType() == ogr.wkbPolygon:
        return union
    #else
    filtered_geom=ogr.Geometry(ogr.wkbMultiPolygon)
    try:
        if union.GetGeometryType() == ogr.wkbMultiPolygon:
            max_index=0
            max_area=0.0
            for i in filter_table[key][1]: #select all in second list.
                geom = union.GetGeometryRef(i)
                #filtered_geom.AddGeometry(geom)
                if geom.GetArea() > max_area:
                    max_area = geom.GetArea()
                    max_index=i
            geom = union.GetGeometryRef(max_index)
            filtered_geom.AddGeometry(geom)
            
    except Exception,e:
        print("Exception in filterCountyRings()"),str(e)
    return filtered_geom


home =r'/home/naresh-1/workspace/machinelrn/data/gis/'
srcshpfile = home + r'uscounties/cb_2016_us_county_500k.shp'
outputShapefile = home+ r'out/states.shp'
state_county_table_file = home+ r'out/state_county_table.p'
filter_table_file = home+ r'out/filter_table.p' #state filter table.
state_dic ={}
state_cc ={}
exclude=['02','66','15','60','78','72','69'] #island states like Alaska,Guam,etc.

#open shape file.
shapef = ogr.Open(srcshpfile)
layer = shapef.GetLayer()
sourceSpatialRef = layer.GetSpatialRef()
#collect county rings for each of the states.
for fid in xrange(layer.GetFeatureCount()):
    fit= layer.GetFeature(fid)
    geom= fit.GetGeometryRef()
    #filter nasty boundaries.
    if fit.GetField('STATEFP') in exclude: #do not include in exclude.
        continue
    try:
        state_cc[fit.GetField('STATEFP')]+=[(fit.GetField('COUNTYFP'),fit.GetField('COUNTYNS'),fit.GetField('NAME'))]
        if geom.GetGeometryType() == ogr.wkbPolygon:
            state_dic[fit.GetField('STATEFP')].AddGeometry(geom)
        elif geom.GetGeometryType() == ogr.wkbMultiPolygon:
            for i in range(geom.GetGeometryCount()):
                gp = geom.GetGeometryRef(i)
                state_dic[fit.GetField('STATEFP')].AddGeometry(gp)
    except:
        state_cc[fit.GetField('STATEFP')]=[(fit.GetField('COUNTYFP'),fit.GetField('COUNTYNS'),fit.GetField('NAME'))]
        state_dic[fit.GetField('STATEFP')]=ogr.Geometry(ogr.wkbMultiPolygon)
        if geom.GetGeometryType() == ogr.wkbPolygon:
            state_dic[fit.GetField('STATEFP')].AddGeometry(geom)
        elif geom.GetGeometryType() == ogr.wkbMultiPolygon:
            for i in range(geom.GetGeometryCount()):
                gp = geom.GetGeometryRef(i)
                state_dic[fit.GetField('STATEFP')].AddGeometry(gp)


#load filter table, if not exist create one.filter table has state-id:[county-rings],[selective-rings]
try:
    filter_table= pickle.load( open(filter_table_file, "rb" ))
    print("filter_table file exists in: "),filter_table_file
except:
    filter_table={}
    for key, unionc in state_dic.items():
        try:
            union= unionc.UnionCascaded()
            filter_table[key]=[]
            if union.GetGeometryType() == ogr.wkbMultiPolygon:
                filter_table[key].append([j for j in range(union.GetGeometryCount())])#original-rings
                filter_table[key].append([j for j in range(union.GetGeometryCount())]) #put-items here to slect.
            else:
                filter_table[key].append([])
                filter_table[key].append([])
        except Exception, ex:
            print("Error while creating filter-table"),key, str(ex)
    
    pickle.dump(filter_table,open(filter_table_file, "wb" )) #end except
    print("filter_table dict. is saved in: "),filter_table_file
    
#save state-county-table.
try:
    state_county_table= pickle.load(open(state_county_table_file,"rb"))
    print("state_county_table exists in pickle file: "),state_county_table_file
except:
    pickle.dump(state_cc, open(state_county_table_file,"wb"))

#modify the filter table
#saving layer
driver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(outputShapefile):
    driver.DeleteDataSource(outputShapefile)

outDataSet = driver.CreateDataSource(outputShapefile)
outLayer = outDataSet.CreateLayer("states",sourceSpatialRef,geom_type=ogr.wkbMultiPolygon)#name,SpatialRef,geom type

outLayer.CreateField(ogr.FieldDefn('STATEFP'),ogr.OFTInteger)
outLayer.CreateField(ogr.FieldDefn('area'),ogr.OFTInteger)

layerDefn = outLayer.GetLayerDefn()
print("key,#counties,#unionc,#union:")    
for key, unionc in state_dic.items():
    try:
        union= unionc.UnionCascaded()        
        print key, len(state_cc[key]),unionc.GetGeometryCount(), union.GetGeometryCount(),union.GetGeometryName()  
        feat = ogr.Feature(layerDefn)
        #filter
        filtered_geoms = filterCountyRings(key,union,filter_table)
        feat.SetGeometry(filtered_geoms)
        feat.SetField('STATEFP',key)
        feat.SetField('area',filtered_geoms.GetArea())
        outLayer.CreateFeature(feat)
        feat.Destroy()
        union.Destroy()
    except Exception, e:
        print("except"),key,str(e)
outDataSet.Destroy()
print("States boundaries saved in "),outputShapefile

filter_table file exists in:  /home/naresh-1/workspace/machinelrn/data/gis/out/filter_table.p
state_county_table exists in pickle file:  /home/naresh-1/workspace/machinelrn/data/gis/out/state_county_table.p
key,#counties,#unionc,#union:
30 56 56 1 POLYGON
54 55 55 1 POLYGON
42 67 68 1 POLYGON
45 46 50 4 MULTIPOLYGON
36 62 83 18 MULTIPOLYGON
53 39 96 58 MULTIPOLYGON
32 17 17 1 POLYGON
24 24 41 18 MULTIPOLYGON
25 14 47 33 MULTIPOLYGON
26 83 193 111 MULTIPOLYGON
01 67 74 7 MULTIPOLYGON
20 105 105 1 POLYGON
21 120 121 2 MULTIPOLYGON
04 15 15 1 POLYGON
49 29 29 1 POLYGON
46 66 66 1 POLYGON
23 16 223 207 MULTIPOLYGON
08 64 78 1 POLYGON
09 8 15 8 MULTIPOLYGON
28 82 89 8 MULTIPOLYGON
29 115 115 1 POLYGON
40 77 77 1 POLYGON
41 36 43 8 MULTIPOLYGON
56 23 23 1 POLYGON
10 3 6 4 MULTIPOLYGON
39 88 99 12 MULTIPOLYGON
47 95 102 1 POLYGON
27 87 91 5 MULTIPOLYGON
06 58 72 13 MULTIPOLYGON
38 53 53 1 POLYGON
11 1 1 1 POLYGON
51 133 149 11 MULTIPOLYGON
13 159 166 2 MULTIPOLYGON
12 67 188 116 MULTIPOLYGON
